In [1]:
from pathlib import Path
import sys

# Define the base repository directory
repo_dir = Path.home() / "Desktop" / "eye_ai" / "Github"  # Update to your GitHub repo location

# Update the load path so Python can find modules for the model
sys.path.insert(0, str(repo_dir / "deriva-ml"))
sys.path.insert(0, str(repo_dir / "eye-ai-ml"))

# Reload extensions if needed
%load_ext autoreload
%autoreload 2


In [2]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
from deriva_ml.dataset_bag import DatasetBag
import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml.deriva_ml_base import MLVocab as vc
from deriva_ml.execution_configuration import ExecutionConfiguration, Workflow, Execution

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'dev.eye-ai.org'
catalog_id = "428"

# host = 'www.eye-ai.org'
# catalog_id = "21"
# catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-11-21 22:43:25,626 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-11-21 22:43:25,628 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [4]:
# Variables to configure the rest of the notebook.
cache_dir = Path.home() / 'Desktop/test_cache'
working_dir = Path.home() / 'Desktop/test_cache'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-11-21 22:43:25,659 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-11-21 22:43:25,659 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-11-21 22:43:26,264 - INFO - Loading dirty model.  Consider commiting and tagging: 1.1.0.post76+git.403fdd90.dirty


# 1. Configuration

In [5]:
# Add Workflow Vocab terms
EA.add_term(vc.workflow_type, "Test Workflow", description="A test Workflow for new DM")
# Workflow instance
test_workflow = Workflow(
    name="Test New Workflow-multimodal",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/schema_changes/template_multimodal.ipynb",
    workflow_type="Test Workflow"
)
# Configuration instance
config = ExecutionConfiguration(
    bdbags=['2-AGAW'], #minid:1JiKxaIcjAIaO
    models = ['2-4JR6'],
    execution=Execution(description="Sample Test Execution"),
    workflow=test_workflow,
    description="Our Test Workflow instance")
# Initialize execution
configuration_record = EA.initialize_execution(config)
execution_rid = configuration_record.execution_rid


2024-11-21 22:45:24,003 - INFO - Configuration validation successful!


DerivaMLException: Failed to insert workflow. Error: [DerivaMLException] Invalid URL: https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/schema_changes/template_multimodal.ipynb

In [17]:
print(configuration_record)
configuration_record.bag_paths[0]

caching_dir: /Users/zhiweili/Desktop/test_cache
working_dir: /Users/zhiweili/Desktop/test_cache/zhiweili/EyeAI_working
execution_rid: 5-R6F6
workflow_rid: 5-R6B4
bag_paths: [PosixPath('/Users/zhiweili/Desktop/test_cache/2-7K8W_d1ead67f8439d2eca28191477fdb771f579043af01ea5817ed2763c9b7b9a2d2/Dataset_2-7K8W')]
asset_paths: [PosixPath('/Users/zhiweili/Desktop/test_cache/zhiweili/EyeAI_working/5-R6F6/models/optic_disk_crop_model.hdf5')]
configuration: bdbags=['2-7K8W'] models=['2-4JR6'] workflow=Workflow(name='Test New Workflow', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/schema_changes/test_eye-ai_changes.ipynb', workflow_type='Test Workflow', version=None, description=None) execution=Execution(description='Sample Test Execution') description='Our Test Workflow instance'


PosixPath('/Users/zhiweili/Desktop/test_cache/2-7K8W_d1ead67f8439d2eca28191477fdb771f579043af01ea5817ed2763c9b7b9a2d2/Dataset_2-7K8W')

In [7]:
ds_bag = DatasetBag(configuration_record.bag_paths[0])

# Test extract image and reshape method

In [26]:
init_diag = EA.image_tall(ds_bag, 'Initial Diagnosis')
grader_diag = EA.image_tall(ds_bag, 'AI_glaucomasuspect_test')
long, wide = EA.reshape_table([init_diag, grader_diag], 'Diagnosis_Image')

# Test extract diagnosis and gold standard calculation

In [30]:
def CDR_func(values):
    return values.mean()

def Diag_func(values):
    # Use the mode function to get the most frequent diagnosis value
    mode_diagnosis = values.mode()

    # If there are multiple modes, take the first one
    if not mode_diagnosis.empty:
        return mode_diagnosis.iloc[0]
    else:
        # Handle the case where there is no mode (e.g., all values are unique)
        return 'Unknown'

def ImageQuality_func(values):
    mode_quality = values.mode()
    if not mode_quality.empty:
        return mode_quality.iloc[0]
    else:
        return "Unknown"
    

In [33]:
grader_list = ["Benjamin Xu", "Kendra Hong", "Kent Nguyen"]
result = EA.compute_diagnosis(grader_diag[grader_diag["Full_Name"].isin(grader_list)],
                                 Diag_func,
                                 CDR_func,
                                 ImageQuality_func)

/Users/zhiweili/Desktop/eye_ai/Github/eye-ai-ml/eye_ai/eye_ai.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Test Crop Image 

In [36]:
output_dir = configuration_record.working_dir
EA.create_cropped_images(configuration_record.bag_paths[0], ds_bag, output_dir, crop_to_eye=True)

(PosixPath('/Users/zhiweili/Desktop/test_cache/zhiweili/EyeAI_working/Image_cropped'),
 PurePosixPath('/Users/zhiweili/Desktop/test_cache/zhiweili/EyeAI_working/Cropped_Image.csv'))

In [ ]:
# with ml_instance.execution(configuration=configuration_record) as exec:
#     output_dir = ml_instance.execution_assets_path / "Feature1"
#     output_dir.mkdir(parents=True, exist_ok=True)
#     with open(output_dir / "test.txt", "w+") as f:
#         f.write("Hello there\n")    

In [37]:
uploaded_assets = EA.upload_execution(configuration_record, True)

2024-11-21 22:29:39,774 - INFO - Initializing uploader: GenericUploader v1.7.4 [Python 3.10.15, macOS-10.16-x86_64-i386-64bit]
2024-11-21 22:29:39,779 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-11-21 22:29:39,780 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-11-21 22:29:39,952 - INFO - Scanning files in directory [/Users/zhiweili/Desktop/test_cache/zhiweili/EyeAI_working/deriva-ml/execution/5-R6F6/execution-assets]...
2024-11-21 22:29:39,953 - INFO - File upload processing completed: 0 files were uploaded successfully, 0 files failed to upload due to errors, 0 files were skipped because they did not satisfy the matching criteria of the configuration.
2024-11-21 22:29:40,414 - INFO - Initializing uploader: GenericUploader v1.7.4 [Python 3.10.15, macOS-10.16-x86_64-i386-64bi